# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os

In [2]:
SEEDS = [10, 42, 4] # don't change!

In [3]:
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### load the data
# if needed, specify batch sizes and shuffle settings
print('==> Loading data..')
dataloader = DataLoader()
dataloader.download_cifar()
trainloader, testloader = dataloader.get_loaders()

### Model
print('==> Building model..')
net = ResNet18()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)
if device == 'cuda':
    net.to(device)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### hyperparameters
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 5 # number of iterations the model gets trained
learning_rate = 1e-4 # factor for weight updates
momentum = 0.9
weight_decay = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

### Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

### Testing
def test(epoch, best_acc, seed):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    print(acc)
    if acc > best_acc:
        print("==> Saving to checkpoint..")
        net.save(acc, epoch, seed)
    return best_acc

==> Loading data..
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


In [5]:
resume = False

for seed in SEEDS:
    print("Seed: {}".format(seed))
    torch.manual_seed(seed)
    
    if resume:
        print('==> Resuming from checkpoint..')
        assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
        ckpt = torch.load('./serialized/ckpt_{}.pth'.format(seed))
        best_acc, start_epoch, net = net.load(ckpt)
    
    for epoch in range(start_epoch, start_epoch+num_epochs):
        train(epoch)
        best_acc = test(epoch, best_acc, seed)

Seed: 10

Epoch: 0
30.38
==> Saving to checkpoint..

Epoch: 1
38.27
==> Saving to checkpoint..

Epoch: 2
43.36
==> Saving to checkpoint..

Epoch: 3
47.22
==> Saving to checkpoint..

Epoch: 4
49.8
==> Saving to checkpoint..
Seed: 42

Epoch: 0
52.89
==> Saving to checkpoint..

Epoch: 1
55.22
==> Saving to checkpoint..

Epoch: 2
56.9
==> Saving to checkpoint..

Epoch: 3
58.77
==> Saving to checkpoint..

Epoch: 4
59.32
==> Saving to checkpoint..
Seed: 4

Epoch: 0
60.06
==> Saving to checkpoint..

Epoch: 1
62.28
==> Saving to checkpoint..

Epoch: 2
62.14
==> Saving to checkpoint..

Epoch: 3
63.75
==> Saving to checkpoint..

Epoch: 4
64.8
==> Saving to checkpoint..
